In [ ]:
from influxdb_client import InfluxDBClient
import pandas as pd

# Configuration
influxdb_url = "http://localhost:8086"
token = "random_token"
org = "ORG"
bucket = "system_state"
labels = "labels_TEMP_seq"
bucket_ano = "anomalies"
dataset = "repad2-temp-result-minmax"
detections = "miura-temp-detection-standard"
margin = 3  # Margin of T values

# Initialize client
client = InfluxDBClient(url=influxdb_url, token=token, org=org)
query_api = client.query_api()

start_time = "1997-04-10T00:00:00Z"

# Construct and fetch data
queries = {
    "labels": f'''
        from(bucket: "{bucket}")
        |> range(start: time(v: "{start_time}"))
        |> filter(fn: (r) => r["_measurement"] == "{labels}")
        |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
    ''',
    "dataset": f'''
        from(bucket: "{bucket_ano}")
        |> range(start: time(v: "{start_time}"))
        |> filter(fn: (r) => r["_measurement"] == "{dataset}")
        |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
    ''',
    "detections": f'''
        from(bucket: "{bucket_ano}")
        |> range(start: time(v: "{start_time}"))
        |> filter(fn: (r) => r["_measurement"] == "{detections}")
        |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
    '''
}
# Query data and store in dictionary
results = {key: query_api.query_data_frame(query=queries[key]) for key in queries}

# Check if any dataset is empty
if any(df.empty for df in results.values()):
    print("One or more datasets are empty. Check data and queries.")
else:
    # Prepare datasets
    for key, df in results.items():
        df['timestamp'] = pd.to_datetime(df['_time'])

    # Align T values with labels and sort by T (Time)
    full_labels = pd.merge(results['dataset'], results['labels'], on='timestamp', how='inner')
    full_labels = full_labels.sort_values(by='T')

    # Create groups based on neighbouring T-values
    full_labels['group'] = (full_labels['T'].diff() != 1).cumsum()

    # Track detection matches for each group
    group_matches = {group_id: False for group_id in full_labels['group'].unique()}

    # Track detection and label matches
    detection_matches = {index: False for index in results['detections'].index}
    label_matches = {index: False for index in full_labels.index}

    # Iterate over each anomaly group and check for detections within the margin
    for group_id, group in full_labels.groupby('group'):
        for index, detection in results['detections'].iterrows():
            if any((detection['T'] >= row['T'] - margin) and (detection['T'] <= row['T'] + margin) for _, row in group.iterrows()):
                detection_matches[index] = True
                group_matches[group_id] = True # Event-wise match
                label_matches.update({idx: True for idx, _ in group.iterrows()}) # Point-wise match


    # Calculate true positives, false positives, and false negatives
    ## Point-wise calculation
    true_positives  = sum(match for match in detection_matches.values())
    false_negatives = sum(not match for match in label_matches.values())
    false_positives = sum(not match for match in detection_matches.values())

    ## Event-wise calculation (False positives are point-wise)
    #true_positives  = sum(match for match in group_matches.values())
    #false_negatives = sum(not match for match in group_matches.values())
    #false_positives = sum(not match for match in detection_matches.values())


    # Metrics calculations
    precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

    # For datasets with no anomalies
    if (true_positives + false_positives + false_negatives) == 0:
        print("No anomalies in dataset, and no detections made")
        precision = 1
        recall = 1
        f1_score = 1
        

    print(f"True Positives: {true_positives}, False Positives: {false_positives}, False Negatives: {false_negatives}")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1_score:.4f}")

    with open(f'../{labels}_{detections}.txt', 'w') as f:
        print(f"True Positives: {true_positives}, False Positives: {false_positives}, False Negatives: {false_negatives}", file=f)
        print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1_score:.4f}", file=f)